# 📌 Was ist eine Loss Function / Verlustfunktion?

Die **Loss Function (Verlustfunktion)** ist das Herzstück des Lernprozesses im Deep Learning.

Die Loss Function misst, wie falsch das Modell aktuell liegt.
**Das Ziel des Trainings ist es, den Loss zu minimieren,** also:

"Die Vorhersagen des Modells sollen **immer näher an die wahren Labels kommen."**
### Binary Entropy Loss ? (BCELoss)
criterion = nn.BCELoss()

loss = criterion(y_pred, y_true)

Pytorch modeul , Standard bei binärer Klassifikation (0 vs. 1)

Verwendete Formel der Binary Cross Entropy:

$$
\text{Loss} = -\frac{1}{n} \sum_{i=1}^{n} \left[ y_i \cdot \log(p_i) + (1 - y_i) \cdot \log(1 - p_i) \right]
$$

---


Mit Mask : 1

Ohne Mask : 0 

$$
L(\hat{o}, (i, o)) \in [0, \infty)
$$

$$
\text{Loss} = - \left[ y \cdot \log(p) + (1 - y) \cdot \log(1 - p) \right]
$$

Dabei ist:
- \( y \in \{0, 1\} \) das echte Label  
- \( p \in [0,1] \) die vom Modell vorhergesagte Wahrscheinlichkeit


# 🧠 Entscheidungsfunktion: Theorie & Definition

Die **Entscheidungsfunktion** sagt aus, welche Ausgabe das Modell für eine bestimmte Eingabe produziert.
Formal gesagt:

Eine Funktion 
𝛿
δ, die jedem Eingabevektor 
𝑖
i aus dem Merkmalsraum 
𝑆
S eine Entscheidung 
𝑜
o aus dem Ausgaberaum 
𝑂
O zuordnet.

$$
\delta : \mathcal{S} \rightarrow O
$$

- \(S ⊆{R}^n\) ist der Merkmalsraum (Input)
- \(O ⊆{R}\) ist der Ausgaberaum (z. B. {0, 1})

Im Kontext der Maskenerkennung:

- Eingabe \(i\): Bilddaten eines Gesichts
- Ausgabe \(δ(i)\): Entscheidung = „Maske“ (1) oder „keine Maske“ (0)

⚠️ Die Ausgabe \(δ(i)\) kann sich von der wahren Ausgabe \(o\) unterscheiden.


**Das bedeutet Zwei Klassen und eine Entscheidung**


**Entscheidungsfunktion (δ)  →  Verlustfunktion (L)  →  BCE-Formel  →  Loss.backward()**


In [2]:
import torch
import torch.nn as nn
from conda.testing.helpers import TEST_DATA_DIR

In [7]:
# All elements of target should be between 0 and 1
predictions = torch.tensor([0.9, 0.2, 0.8], dtype=torch.float32) # Beispiel: Modellvorhersagen (Wahrscheinlichkeiten)
targets = torch.tensor([1.0, 0.0, 0.3], dtype=torch.float32)     # Echte Labels

In [8]:
loss_fc = nn.BCELoss()      # Binary Entropy Loss
loss = loss_fc(predictions, targets)   # Loss Berechnen
print(f"Loss: {loss.item():.4f}")

Loss: 0.5074


Ein eigenes Modell (z. B. MyCNN)

Datenvorbereitung + DataLoader

Loss-Funktion (BCELoss)

Optimizer (z. B. Adam)

Trainingsschleife

In [24]:
# 1- MyCNN
class MyCNN(nn.Module):
    def __init__(self):                               # Netzwerk definiert und initialisiert
        super(MyCNN, self).__init__()                 # Ruft den Konstruktor der Basisklasse nn.Module auf. Das ist nötig, damit PyTorch intern alles korrekt registriert.
        # Convolutional Layer-Block
        self.conv_layer = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),  # Erste Convolution-Schicht. Input hat 1 Kanal (Graustufenbild), Output 16 Filter.
            nn.ReLU(),                                             # Aktivierungsfunktion – führt Nichtlinearität ein.
            nn.MaxPool2d(2),                                       # Verkleinert die Bildgröße um die Hälfte.
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1), nn.ReLU(), nn.MaxPool2d(2),  # Zweite Convolution – jetzt 32 Filter.
            nn.ReLU(),                                               # Aktivierungsfunktion – führt Nichtlinearität ein. warum 2te mal
            nn.MaxPool2d(2)                                          # Verkleinert die Bildgröße um die Hälfte. warum 2te mal
        )
        # Fully Connected Layer
        self.fc_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(288, 1),
            nn.Sigmoid()  # Wichtig für BCE!
        )
    # DAMIT DAS MANN KANN RUFT JEDES MAL BATCH   x output  
    def forward(self, x):
        x = self.conv_layer(x)
        x = self.fc_layer(x)
        return x    

### Was macht Fully Connected Layer (FC)? Kann es ohne Konvertierung verwendet werden?
Seine Funktion: Er fasst alle Merkmale in einem einzigen Vektor zusammen und trifft die endgültige Entscheidung (zum Beispiel: „Ist eine Maske vorhanden?“ → ja/nein).

Mathematisch: Jedes Neuron ist mit allen Neuronen in der vorherigen Schicht verbunden.

✅ Ja, es kann ohne Faltung verwendet werden (z. B. MLP – Multilayer Perceptron), aber es ist für Bilder sehr ineffizient, weil:

Positionsinformationen gehen verloren.

Zu viele Parameter → Überanpassung.

In [25]:
from torch.utils.data import DataLoader, TensorDataset

# Dummy-Daten: 100 Bilder in Grau (1 Kanal, 28x28)
X = torch.randn(100, 1, 28, 28)
y = torch.randint(0, 2, (100, 1)).float()  # Binäre Labels (0 oder 1)

dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=16, shuffle=True)


In [26]:
# TRAINING
model = MyCNN()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(5):
    for batch_X, batch_y in loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 0.7857
Epoch 2, Loss: 0.6931
Epoch 3, Loss: 0.6740
Epoch 4, Loss: 0.6722
Epoch 5, Loss: 0.6950


# 🧠 Erwartetes Verlustrisiko (Expected Risk)Das Ziel eines ML-Modells ist, eine Funktion 
Das Ziel eines ML-Modells ist, eine Funktion 
𝛿
δ zu finden, die bei gegebenen Eingabedaten 
𝑖
i eine gute Vorhersage 
𝑜
o macht. Dazu definieren wir:

Die Verlustfunktion \( L(\hat{o}, (i, o)) \) misst den Fehler zwischen der Vorhersage \( \hat{o} \) und dem tatsächlichen Wert \( o \), gegeben einer Eingabe \( i \). Da die Eingaben aus einer Verteilung stammen, betrachten wir den Erwartungswert des Verlusts über alle möglichen Eingaben:

$$
R(\delta) := \int_{\Omega} L(\hat{o}, (i, o)) \cdot p(\Omega)
$$

- \(R(δ)): Raum der Eingabe-Ausgabe-Paare \( (i, o) \)
- \(L(⋅)): Verlustfunktion, (wie stark weicht die Vorhersage von der Wahrheit ab)
- \(p(Ω): Wahrscheinlichkeitsdichte für das Auftreten von \( (i, o) \)

👉 Ziel ist es, eine Entscheidungsfunktion \( \delta \) zu finden, die das **Risiko** \( R(\delta) \) minimiert:

$$
\delta^* := \operatorname{argmin}_\delta R(\delta)
$$

Man nennt \(δ* ) auch die **Bayes'sche Entscheidungsfunktion**, da sie unter der gegebenen Verteilung den geringsten erwarteten Verlust erzielt.


### Was sind ReLU und MaxPool? Warum MaxPool2d?
    🧠 ReLU (gleichgerichtete lineare Einheit):
            Aktivierungsfunktion: Macht aus negativen Werten 0.

            Die Berechnung ist sehr schnell.

    Es bringt Nichtlinearität. Andernfalls kann das Modell nicht über das „Zeichnen einer Linie“ hinausgehen.

    🧱 MaxPool2d:
            Es komprimiert die Daten, reduziert also die Größe.
            
            Nimmt 2x2 Regionen und behält nur den größten Wert.
            
            Reduziert Lärm und ermöglicht effizientes Lernen.

### Warum 2D?

Bilder sind 2D (Höhe × Breite), daher verwenden wir MaxPool2d.

            MaxPool1d → für Zeitreihen
            
            MaxPool3d → für 3D-Daten (z. B. Videos)

### Warum gibt es nach jeder Faltung ReLU + MaxPool?
            Bei der Faltung werden nur Filter angewendet, jedoch keine Nichtlinearität oder Dimensionsreduzierung erreicht.
            
            ReLU → stärkt das Lernen (vermeidet Gradientenverlust).
            
            MaxPool → fasst Funktionen zusammen und bietet eine bessere Generalisierung mit weniger Parametern.
            
            Jedes Mal, wenn dieses Paar angewendet wird, gilt für das Modell:
            
            Lernt besser.
            
            Kann mit weniger Daten verallgemeinern.
            
            Die Berechnung wird effizienter.
            
# Lernverfahren

**1. Perzeptron-Ausgabe (einfaches Neuron mit Aktivierungsfunktion):**
$$
\phi(x) = \rho\left( \sum_{j=1}^{n} x_j \cdot w_{j,i} + b \right) = x_i
$$

**2. Gewichtsanpassung im Hopfield-Netzwerk (Hebbsche Lernregel):**
$$
w(i,j) = \sum_{k=1}^{n} a_k(i) \cdot a_k(j)
$$

**3. Neuron-Ausgabe in Layer \( l \) eines Feedforward-Netzes (mit Bias und Aktivierungsfunktion):**
$$
x_h^{(l)} = \rho^{(l)}\left( \sum_{j=1}^{n^{(l-1)}} x_j^{(l-1)} \cdot w_{j,h}^{(l)} + b_h^{(l)} \right)
$$

**4. Backpropagation – Gewichtsanpassung (Gradient Descent Regel):**
$$
\Delta w_{j,h}^{(l)} = \eta \cdot \frac{\partial L}{\partial w_{j,h}^{(l)}}
$$

**5. Backpropagation – Bias-Anpassung:**
$$
\Delta b_h^{(l)} = \eta \cdot \frac{\partial L}{\partial b_h^{(l)}}
$$

**6. Neue Gewichte nach der Aktualisierung (Gewichtsupdate):**
$$
w_{j,h}^{(l),\text{neu}} = w_{j,h}^{(l),\text{alt}} - \Delta w_{j,h}^{(l)}
$$

**7. Neue Bias-Werte nach der Aktualisierung:**
$$
b_h^{(l),\text{neu}} = b_h^{(l),\text{alt}} - \Delta b_h^{(l)}
$$

**8. Beispiel: Änderung eines Gewichts mit Lernrate \( \eta = 0.1 \):**
$$
\Delta w_1 = \eta \cdot \frac{\partial L}{\partial w_1} = 0.1 \cdot (-0.5) = -0.05
$$

**9. Neues Gewicht nach Update:**
$$
w_1^{\text{neu}} = w_1 + \Delta w_1 = 0.5 - 0.05 = 0.45
$$

**10. Zweites Beispiel mit positivem Gradienten:**
$$
\Delta w_3 = 0.1 \cdot (+0.05) = +0.005 \quad \Rightarrow \quad w_3^{\text{neu}} = -0.05 + 0.005 = -0.045
$$

**11. Allgemeine Gewichtsanpassungsformel (Gradientenverfahren):**
$$
w_{\text{neu}} = w_{\text{alt}} - \eta \cdot \nabla L
$$

